In [63]:
# spot check machine learning algorithms on the adult imbalanced dataset
from numpy import mean
from numpy import std
import pdb
import pandas as pd
from pandas import read_csv
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def load_dataset(full_path):
    # load the dataset as a numpy array
    # dataframe = read_csv(full_path, header=None, na_values='?')
    dataframe = read_csv(full_path, na_values='?')
    # drop rows with missing
    dataframe = dataframe.dropna()

    # split into inputs and outputs
    # last_ix = len(dataframe.columns) - 1
    last_ix = len(dataframe.columns)-1
    X, y = dataframe.drop(index=last_ix, axis=1), dataframe.iloc[:,-1]

    # select categorical and numerical features
    cat_ix = X.select_dtypes(include=['object', 'bool']).columns
    num_ix = X.select_dtypes(include=['int64', 'float64']).columns
    c_ix = []
    n_ix = []
    for i, v in enumerate(dataframe.columns.tolist()):
        if v in cat_ix:
            c_ix.append(i)
        elif v in num_ix:
            n_ix.append(i)
    # label encode the target variable to have the classes 0 and 1
    
#     y = pd.to_numeric(y.values[:,0])
#     y = LabelEncoder().fit_transform(y)
    return X.values, y, c_ix, n_ix

# evaluate a model
def evaluate_model(X, y, model, X_not_san, y_not_san):
    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X_not_san, y_not_san, test_size=0.20, random_state=42)
    
    # define evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
    
    # evaluate model
    model.fit(X, y)
#     model.inverse_transform()
#     scores = cross_val_score(model, X_not_san, y_not_san, scoring='accuracy', cv=cv, n_jobs=2)
#     X_not_san = model.transform(X_not_san)
    preds = model.predict(X_not_san)
    
    score = accuracy_score(y_test,preds)
    
    return score

# define models to test
def get_models():
    models, names = list(), list()
    # CART
    models.append(DecisionTreeClassifier())
    names.append('CART')
    # SVM
#     models.append(SVC(gamma='scale'))
#     names.append('SVM')
#     # Bagging
#     models.append(BaggingClassifier(n_estimators=100))
#     names.append('BAG')
#     # RF
#     models.append(RandomForestClassifier(n_estimators=100))
#     names.append('RF')
#     # GBM
#     models.append(GradientBoostingClassifier(n_estimators=100))
#     names.append('GBM')
    return models, names


# file = 'train_sex'
file = 'adult_sanitized_0.9875'
# define the location of the dataset
full_path = f'../focus_data/gansanitized/{file}.csv'
# load the dataset
X, y, cat_ix, num_ix = load_dataset(full_path)

original_data = f"../GeneralDatasets/Csv/Adult_NotNA_.csv"
# original_data = f"../focus_data/2_sex.csv"
# X_not_san, y_not_san, cat_ix_not_san, num_ix_not_san = load_dataset(original_data)


# define models
models, names = get_models()
results = list()
total_texts = []
# evaluate each model

for i in range(len(models)):
    # define steps
    steps = [('c',OneHotEncoder(handle_unknown='ignore'),cat_ix), ('n',MinMaxScaler(),num_ix)]
    # one hot encode categorical, normalize numerical
    ct = ColumnTransformer(steps)
    # wrap the model i a pipeline
    pipeline = Pipeline(steps=[('t',ct),('m',models[i])])

    # evaluate the model and store results
    scores = evaluate_model(X,y, pipeline,X_not_san, y_not_san)
    results.append(scores)
    # summarize performance
    print_text = "{},{},{}".format(names[i], mean(scores), std(scores))
    total_texts.append(print_text)
    print(print_text)

# plot the results
# pyplot.boxplot(results, labels=names, showmeans=True)
# pyplot.savefig(f"./experiments/metadata_{file}.png")
# Write the results to file

ValueError: Found input variables with inconsistent numbers of samples: [4521, 4522]

In [24]:
total_score = 0
for result in total_texts:
    score = result.split(',')[1]
    total_score = total_score + float(score)
average = total_score/len(total_texts)



with open(f"./experiments/train-{file}_test-{str(original_data).split('/')[-1][:-4]}txt", 'w+') as f:
            f.write(f"Results: {results} \n \n")
            f.write(f"Test Output: {total_texts}\n")
            f.write(f"Classifiers average:{average}")

In [3]:
sv =SVC()

In [55]:
original_data = f"../GeneralDatasets/Csv/Adult_NotNA_.csv"
df = pd.read_csv(original_data)

In [56]:
cat_ix = df.select_dtypes(include=['object', 'bool']).columns
num_ix = df.select_dtypes(include=['int64', 'float64']).columns
c_ix = []
n_ix = []
for i, v in enumerate(df.columns.tolist()):
    if v in cat_ix:
        c_ix.append(i)
    elif v in num_ix:
        n_ix.append(i)
oh = OneHotEncoder(c_ix, handle_unknown='ignore')
mms = MinMaxScaler(n_ix)

In [58]:
oh.fit(df.to_numpy())
mms.fit(df.to_numpy())

ValueError: Shape mismatch: if categories is an array, it has to be of shape (n_features,).

In [26]:
df.to_numpy()

array([[39, 'State-gov', 77516, ..., 40, 'United-States', 0],
       [50, 'Self-emp-not-inc', 83311, ..., 13, 'United-States', 0],
       [38, 'Private', 215646, ..., 40, 'United-States', 0],
       ...,
       [38, 'Private', 374983, ..., 50, 'United-States', 0],
       [44, 'Private', 83891, ..., 40, 'United-States', 0],
       [35, 'Self-emp-inc', 182148, ..., 60, 'United-States', 1]],
      dtype=object)

In [27]:
a = np.load('../laftr/data/adult/adult.npz')
d = dict(zip(("data1A","data1B","data1C"), (a[k] for k in a)))
d['data1A'].shape

NameError: name 'np' is not defined